In [46]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf

%matplotlib inline
%load_ext autoreload
%autoreload 2s

np.set_printoptions(threshold=np.nan)

os.chdir(r'C:\Users\visha\Documents\try')
from fr_utils import *
from inception_blocks_v2 import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# pre-processing images

from scipy import misc
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
DIR=r'C:\Users\visha\Documents\try\face_trim'
l=os.listdir(DIR)
i=0
gg=[]
images=[]
print(l)
for img in l:
   s=os.path.join(DIR,img)
   os.chdir(s)
   cwd=os.getcwd()
   print(cwd)
   k=os.listdir(s)
   for j in k:
       w=os.path.join(s,j)
       images.append(cv2.imread(w))
       try:
           images[i]=cv2.resize(images[i],(96,96))
           i+=1
       except:
           print('failed resizing image %s'%(w))
           images=images[:-1]
        
       #plt.imshow(images[i])
       #plt.show()
       print(i)
       y=np.array(images[i-1])
       if(y.shape!=(96,96,3)):
           print(w+" "+str(y.shape))
           images=images[:-1]
           gg.append(w)
           i-=1
images=np.asarray(images)
images=images.astype(float)
print("i:"+str(i))
print(images.shape)
for x in range(3):
    print(np.mean(images[:,:,x]))
    print(np.std(images[:,:,x]))
for x in range(3):
   images[:,:,x]=np.subtract(images[:,:,x],np.mean(images[:,:,x]))
   images[:,:,x]=np.divide(images[:,:,x],np.std(images[:,:,x]))

print(images.shape)

['nm', 'sk', 'vk']
C:\Users\visha\Documents\try\face_trim\nm
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
C:\Users\visha\Documents\try\face_trim\sk
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
C:\Users\visha\Documents\try\face_trim\vk
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
i:152
(152, 96, 96, 3)
123.537669042
87.2116635912
123.061700475
86.1353357256
122.114149306
86.0280675869
(152, 96, 96, 3)


In [16]:
x=np.transpose(images,(0,3,1,2))
#anchor=images[0:32,:,:,:]
#positive=images[32:64,:,:,:]
#negative=images[64:80,:,:,:]
#negative=np.append(negative,images[140:156,:,:,:],axis=0)
#anchor=np.append(anchor,images[64:102,:,:,:],axis=0)
#positive=np.append(positive,images[102:140,:,:,:],axis=0)
#negative=np.append(negative,images[0:19,:,:,:],axis=0)
#negative=np.append(negative,images[140:159,:,:,:],axis=0)
#anchor=np.append(anchor,images[140:183,:,:,:],axis=0)
#positive=np.append(positive,images[183:226,:,:,:],axis=0)
#negative=np.append(negative,images[0:21,:,:,:],axis=0)
#negative=np.append(negative,images[32:54,:,:,:],axis=0)
#print(anchor.shape)
#print(positive.shape)
#print(negative.shape)
#x=[anchor,positive,negative]

In [17]:
os.chdir(r'C:\Users\visha\Documents\try')

In [27]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [28]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    
    anchor=y_pred[0:22,:]
    positive=y_pred[22:44,:]
    negative=y_pred[44:55,:]
    negative=tf.concat([negative, y_pred[102:113,:]], axis=0)
    
    anchor=tf.concat([anchor,y_pred[44:73,:]],axis=0)
    positive=tf.concat([positive,y_pred[73:102,:]],axis=0)
    negative=tf.concat([negative,y_pred[0:14,:]],axis=0)
    negative=tf.concat([negative,y_pred[102:117,:]],axis=0 )
    
    
    anchor=tf.concat([anchor,y_pred[102:127,:]],axis=0)
    positive=tf.concat([positive,y_pred[127:152,:]],axis=0)
    negative=tf.concat([negative,y_pred[0:12,:]],axis=0)
    negative=tf.concat([negative,y_pred[44:57,:]],axis=0)
    
    k=anchor
    anchor=tf.concat([anchor,positive],axis=0)
    positive=tf.concat([positive,k],axis=0)
    
    negative=tf.concat([negative,y_pred[55:66,:]],axis=0)
    negative=tf.concat([negative,y_pred[113:124,:]],axis=0)
    
    negative=tf.concat([negative,y_pred[14:28,:]],axis=0)
    negative=tf.concat([negative,y_pred[117:132,:]],axis=0)
    
    negative=tf.concat([negative,y_pred[12:24,:]],axis=0)
    negative=tf.concat([negative,y_pred[57:70,:]],axis=0)
    
    #anchor,positive,negative=y_pred[0],y_pred[1],y_pred[2]
    #anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    basic_loss =tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss

In [29]:
y=np.ones([152,128])
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

In [30]:
load_weights_from_FaceNet(FRmodel)

In [117]:
FRmodel.fit( x=x, y=y, epochs=100, batch_size=152)

Epoch 1/100
152/152 [==============================] - 30s 195ms/step - loss: 49.1876 - acc: 0.0066
Epoch 2/100
152/152 [==============================] - 36s 240ms/step - loss: 45.6077 - acc: 0.0000e+00
Epoch 3/100
152/152 [==============================] - 38s 252ms/step - loss: 36.7953 - acc: 0.0000e+00
Epoch 4/100
152/152 [==============================] - 37s 246ms/step - loss: 32.4335 - acc: 0.0000e+00
Epoch 5/100
152/152 [==============================] - 38s 253ms/step - loss: 37.7180 - acc: 0.0000e+00
Epoch 6/100
152/152 [==============================] - 38s 253ms/step - loss: 31.0739 - acc: 0.0000e+00
Epoch 7/100
152/152 [==============================] - 39s 254ms/step - loss: 33.3025 - acc: 0.0000e+00
Epoch 8/100
152/152 [==============================] - 40s 262ms/step - loss: 26.6193 - acc: 0.0000e+00
Epoch 9/100
152/152 [==============================] - 39s 255ms/step - loss: 31.9604 - acc: 0.0000e+00
Epoch 10/100
152/152 [==============================] - 39s 255ms/st

Epoch 80/100
152/152 [==============================] - 39s 259ms/step - loss: 31.8352 - acc: 0.0000e+00
Epoch 81/100
152/152 [==============================] - 38s 253ms/step - loss: 29.8474 - acc: 0.0000e+00
Epoch 82/100
152/152 [==============================] - 40s 262ms/step - loss: 30.6055 - acc: 0.0000e+00
Epoch 83/100
152/152 [==============================] - 37s 244ms/step - loss: 31.5477 - acc: 0.0000e+00
Epoch 84/100
152/152 [==============================] - 39s 254ms/step - loss: 31.6354 - acc: 0.0000e+00
Epoch 85/100
152/152 [==============================] - 38s 249ms/step - loss: 30.9436 - acc: 0.0000e+00
Epoch 86/100
152/152 [==============================] - 38s 247ms/step - loss: 31.3740 - acc: 0.0000e+00
Epoch 87/100
152/152 [==============================] - 37s 246ms/step - loss: 29.7472 - acc: 0.0000e+00
Epoch 88/100
152/152 [==============================] - 38s 249ms/step - loss: 30.9797 - acc: 0.0000e+00
Epoch 89/100
152/152 [==============================] -

In [139]:
database = {}
database["modi"] = img_to_encoding("images/modi.jpg", FRmodel)
database["khan"] = img_to_encoding("images/khan_test.jpg", FRmodel)
#database["aamir"] = img_to_encoding("images/aamir_test.jpg", FRmodel)
#database["master_blaster"] = img_to_encoding("images/master_blaster_test.jpg", FRmodel)
database["kohli"] = img_to_encoding("images/kohli.jpg", FRmodel)
#database["bachan"] = img_to_encoding("images/bachan.jpg", FRmodel)
#database["rajan"] = img_to_encoding("images/rajan.jpg", FRmodel)
#database["papon"] = img_to_encoding("images/papon.jpg", FRmodel)
#database["akshay"] = img_to_encoding("images/akshay.jpg", FRmodel)
#database["rahul"] = img_to_encoding("images/rahul.jpg", FRmodel)

In [140]:

def who_is_it(image_path, database, model):
    
    encoding = img_to_encoding(image_path, model)
    min_dist = 100
    
    for (name, db_enc) in database.items():
        
        dist =  np.linalg.norm(encoding-db_enc)
        print(dist)
        if dist < min_dist:
            min_dist =  dist
            identity = name

    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [142]:
who_is_it("images/kohli_test.jpg", database, FRmodel)

0.0125844
0.0163441
0.00979004
it's kohli, the distance is 0.00979004


(0.0097900368, 'kohli')